# RESEGMENTATION PIPELINE

In [1]:
import SRC_RESEG.crop_image as crop_image


## Set up source and destination folders

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import tifffile
from shapely.geometry import Polygon, box

In [ ]:
ddir = 'G:\\Alexis_data\\Project\\SpatialOmics_cellpose\\results'
base_path = "G:\Alexis_data\Project\SpatialOmics_cellpose\data5k\output-XETG00245__0041374__Section_1_1886OD__20250326__175738"

# Select first and third hannels for segmentation (can only select two) 
fullres_chanel_0 = f"{base_path}\morphology_focus\morphology_focus_0000.ome.tif"   
fullres_chanel_2 = f"{base_path}\morphology_focus\morphology_focus_0002.ome.tif"
sample_name = "1886od"

# Levels: pixelsize in µm https://kb.10xgenomics.com/hc/en-us/articles/11636252598925
scalefactors = {
    0: 0.2125,
    1: 0.4250,
    2: 0.85,
    3: 1.7,
    4: 3.4,
    5: 6.8,
    6: 13.6,
    7: 27.2,
}

# Define resolution level and crop size in pixels
level = 0
crop_size = 600/pixel_size = scalefactors[level] 

SyntaxError: cannot assign to expression (4155703933.py, line 23)

## Define the region of interest (ROI)

In [ ]:
ROI_filepath = f"{base_path}\{sample_name}_coords.csv"

# Read the ROI coordinates from the CSV file
plot_roi(ROI_filepath)

In [ ]:
masked_image, mask = create_masked_image(image_chanel_0, x_coords, y_coords, pixelsize)
visualize_masking(image_chanel_0, mask, masked_image)